In [1]:
import torch

In [2]:
!ls

hymenoptera_data.zip  sample_data


In [3]:
ZIPFILE_PATH = "/content/hymenoptera_data.zip"

In [8]:
from zipfile import ZipFile

In [10]:
with ZipFile(ZIPFILE_PATH) as f:
    f.extractall()

In [11]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [12]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
DATA_DIR = "/content/hymenoptera_data"

In [14]:
data_transforms = {
    "train": transforms.Compose([
                                torchvision.transforms.RandomResizedCrop(224),
                                torchvision.transforms.RandomHorizontalFlip(),
                                torchvision.transforms.ToTensor()
    ]),
    "val": transforms.Compose([
                                torchvision.transforms.Resize(256),
                                torchvision.transforms.CenterCrop(224),
                                torchvision.transforms.ToTensor()
    ])
}

In [15]:
data_transforms

{'train': Compose(
     RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
     RandomHorizontalFlip(p=0.5)
     ToTensor()
 ),
 'val': Compose(
     Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
     CenterCrop(size=(224, 224))
     ToTensor()
 )}

In [16]:
image_datasets = {
    'train' : datasets.ImageFolder(os.path.join(DATA_DIR, 'train'), transform = data_transforms['train']),
    'val' : datasets.ImageFolder(os.path.join(DATA_DIR, 'val'), transform = data_transforms['val'])
}

In [17]:
dataLoders = {
    'train' : torch.utils.data.DataLoader(image_datasets['train'], batch_size= 4, shuffle=True, num_workers=4),
    'val' : torch.utils.data.DataLoader(image_datasets['val'], batch_size= 4, shuffle=True, num_workers=4)
}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [18]:
data_sizes= {
    'train': len(image_datasets['train']) ,
    'val': len(image_datasets['val'])
}

In [19]:
data_sizes

{'train': 244, 'val': 153}

In [20]:
class_name = image_datasets['train'].classes

In [21]:
class_name

['ants', 'bees']

In [22]:
model_conv = torchvision.models.resnet34(pretrained=True)
print(model_conv)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 166MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
# Frezing the layers

for param in model_conv.parameters():
    param.requires_grad= False

In [24]:
model_conv.fc

Linear(in_features=512, out_features=1000, bias=True)

In [25]:
num_features = model_conv.fc.in_features

num_features

512

In [26]:
no_of_classes = len(class_name)

model_conv.fc = nn.Linear(num_features, no_of_classes)

In [27]:
type(model_conv)

torchvision.models.resnet.ResNet

In [28]:
model_conv.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_conv.fc.parameters(), lr= 1e-4)

In [32]:
exp_lr_schedular = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.7)

In [33]:
import copy

In [34]:
def training_loop(dataloder, model, loss_fn, optimizer, schedular, no_epochs = 20):

    best_model_wights = copy.deepcopy(model.state_dict())
    best_acc =0.0

    for epoch in range(no_epochs):
        print(f"\t\tEpoch ------- {epoch+1} ------")

        for phase in ['train','val']:
            training = phase == 'train'
            if training:
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_correct = 0

            for inputs, labels in dataloder[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(training):
                    outputs = model(inputs)
                    _ , preds = torch.max(outputs, 1)

                    loss = loss_fn(outputs, preds)

                    if training:
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_correct += torch.sum(preds == labels.data)

            if training:
                schedular.step()

            epoch_loss = running_loss / data_sizes[phase]
            epoch_acc = running_correct.double() / data_sizes[phase] # mod oparetion

            print(f"{phase}, Loss: {epoch_loss}, accuracy: {epoch_acc}")


            if not training and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wights = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wights)

    return model

In [35]:
model = training_loop(dataLoders, model_conv, loss_fn, optimizer, exp_lr_schedular,no_epochs=30)

		Epoch ------- 1 ------
train, Loss: 0.0988488067552203, accuracy: 0.49590163934426235
val, Loss: 0.020253981506220656, accuracy: 0.542483660130719
		Epoch ------- 2 ------
train, Loss: 0.013227353361053545, accuracy: 0.49590163934426235
val, Loss: 0.0072885194513338065, accuracy: 0.542483660130719
		Epoch ------- 3 ------
train, Loss: 0.007305256087241359, accuracy: 0.49590163934426235
val, Loss: 0.00542727186766094, accuracy: 0.542483660130719
		Epoch ------- 4 ------
train, Loss: 0.004370212142706894, accuracy: 0.49590163934426235
val, Loss: 0.0025828938418200388, accuracy: 0.542483660130719
		Epoch ------- 5 ------
train, Loss: 0.0030701296064300373, accuracy: 0.49590163934426235
val, Loss: 0.0023198440333134403, accuracy: 0.542483660130719
		Epoch ------- 6 ------
train, Loss: 0.0023347784566586133, accuracy: 0.49590163934426235
val, Loss: 0.0019783266215871263, accuracy: 0.542483660130719
		Epoch ------- 7 ------
train, Loss: 0.0018980067307282178, accuracy: 0.49590163934426235
